(latex macros)

$$
\renewcommand{\l}{\lambda}
\newcommand{\mtype}[1]{\mathbb{#1}}
\newcommand{\bind}{\star}
\newcommand{\unit}{\eta}
\newcommand{\push}{{\triangleright}}
\newcommand{\nil}{\varepsilon}
\newcommand{\low}{{\Downarrow}}
\renewcommand{\ilow}{\overset{\raise{-3pt}{\pt\Rule{5pt}{0.5pt}{0pt}}}{\Downarrow}}
\newcommand{\lift}{{\uparrow}}
\newcommand{\ilift}{\text{\rotatebox[origin=c]{90}{${\Mapsto}$}}}
\newcommand{\reset}{{\downharpoonleft}\!{\upharpoonright}\!}
\newcommand{\pair}[2]{\tup{#1, #2}}
\newcommand{\thickpair}[2]{\TUP*{#1,\ \ #2}}
\newcommand{\typepair}[2]{#1 \ast #2}
\newcommand{\set}[1]{\left\{#1\right\}}
\newcommand{\p}[1]{\left(#1\right)}
\newcommand{\hole}{[\,]}
\newcommand{\fsl}{\,\big/\!\!\big/\,}
\newcommand{\bsl}{\,\big\backslash\!\!\big\backslash\,}
\newcommand{\msl}{\,\,\big|\hspace{0.5pt}\big|\,\,}
\newcommand{\FSL}[1]{\ \stretchrel[600]{\ \fsl\ }{#1}}
\newcommand{\BSL}[1]{\ \stretchrel[600]{\ \bsl\ }{#1}}
\newcommand{\MSL}[1]{\ \stretchrel[600]{\ \msl\ }{#1}}
\newcommand{\coloneq}{:=}
\newcommand{\dblcolon}{::}
\newcommand{\pt}{\hspace{1pt}}
\newcommand{\npt}{\hspace{-1pt}}
\newcommand{\ppt}{\hspace{2pt}}
\newcommand{\nppt}{\hspace{-2pt}}
\newcommand{\col}{\pt{:}\ppt}
\newcommand{\dt}{\pt{.}\ppt}
\newcommand{\cn}[1]{\textsf{#1}}
\newcommand{\cat}{\hspace{-2pt}\cdot\hspace{-2pt}}
\newcommand{\sto}{\mathbin{\rightarrow}}
\newcommand{\must}{\mathlarger{\mathlarger{\Box}}}
\newcommand{\rest}[2]{#1 \col #2}
\newcommand{\objl}[1]{`#1'}
\newcommand{\tru}{\textsf{\bfseries T}}
\newcommand{\fals}{\textsf{\bfseries F}}
\newcommand{\ms}[1]{\mathsmaller{#1}}
\newcommand{\dom}[1]{\mathcal{D}_{#1}}
\newcommand{\defeq}{\coloneq}
\newcommand{\one}{\textbf{1}}
\newcommand{\M}{\text{\sffamily\bfseries M}}
\newcommand{\N}[1]{\textbf{#1}}
\newcommand{\Only}{\text{\sffamily\bfseries O}}
\renewcommand{\O}{\mathbb{O}}
\newcommand{\Sup}{\text{\sffamily\bfseries S}}
\renewcommand{\S}{\mathbb{S}}
\newcommand{\pl}[1]{\prescript{*}{}{#1}}
\newcommand{\prt}{\sqsubset}
\newcommand{\prteq}{\sqsubseteq}
\newcommand{\nprt}{\not\sqsubset}
\newcommand{\nprteq}{\not\sqsubseteq}
\newcommand{\st}{,\ }
\renewcommand{\leadsto}{\rightsquigarrow}
\newcommand{\reducesto}{\ \rightsquigarrow\ \ }
\newcommand{\supstack}[1]{\subarray{l}#1\endsubarray}
\newcommand{\midstrut}{\setlength\bigstrutjot{7pt}\bigstrut}
\newcommand{\lilstrut}{\setlength\bigstrutjot{2pt}\bigstrut}
\newcommand{\biggstrut}{\setlength\bigstrutjot{10pt}\bigstrut}
$$

In [1]:
{-# LANGUAGE FlexibleInstances #-}
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE TypeFamilies #-}
{-# LANGUAGE TypeOperators #-}

In [2]:
module Grammar where

In [3]:
import Control.Monad.State

## Types

### Type Constructors

Unary types include entities $e$, truth values $t$, and discourse contexts
$\sigma$ (modeled here as lists).
Constructed types take one of the following forms, where $\alpha$ and
$\beta$ are any two types:

* $\alpha \sto \beta$, the type of a function from $\alpha$ to $\beta$.
* $\set{\alpha}$, the type of a set of $\alpha$ objects.
* $\typepair{\alpha}{\beta}$, the type of an $\alpha$ object paired with
  a $\beta$ object, in that order.

### Type Abbreviations
 
To keep type descriptions readable, I use the following type synonyms:

* $\mtype{D}\,{\alpha} \equiv \sigma\sto \set{\typepair{\alpha}{\sigma}}$,
  the type of updates corresponding to constituents of type $\alpha$.
* $\mtype{K}\,{\rho}\,{\omicron}\,{\alpha} \equiv \p{\alpha\sto \omicron}\sto
  \rho$, the type of generalized quantifiers with base type
  $\alpha$.
* $\mtype{F}\,{\alpha} \equiv \mtype{D}\,{\alpha}\sto \mtype{D}\,{\alpha}$,
  the type of filters on updates.

In [4]:
type E = String
type T = Bool

type Stack = [E]

type D = StateT Stack []
type F a = D a -> D a
data K r o a = Tower {runTower :: (a -> o) -> r}

### Modes of combination

#### Recursive forward application

$
\begin{aligned}
m \fsl n
&\coloneq
\begin{cases}
  m\,n \quad 
  \textbf{if}\ m \dblcolon \alpha\sto\beta\st n \dblcolon \alpha \\
  \l k \dt m\,\p{\l f \dt n\,\p{\l x \dt k\,\p{f \fsl x}}} \quad
  \textbf{otherwise}
\end{cases}
\end{aligned}
$

In [5]:
type family f :/: g where
  (a -> b) :/: a = b
  (K c d f) :/: (K d e x) = K c e (f :/: x)

infixr 9 ~/~
class AppR f g where
  (~/~) :: f -> g -> f :/: g
instance AppR (a -> b) a where
  f ~/~ x = f x
instance (d ~ d', AppR in1 in2) => AppR (K c d in1) (K d' e in2) where
  mf ~/~ mx = Tower $ \k -> runTower mf (\f -> runTower mx (\x -> k (f ~/~ x)))

`:/:` is a type-level function that takes two types, `f` and `g`, and returns
the type that would result from combining something of type `f` on the left with something
of type `g` on the right

And `~/~` is the polymorphic function that "applies" a function-y thing on the left
to an argument-y thing; it is // above

#### Recursive backward application

$
\begin{aligned}
m \bsl n
&\coloneq
\begin{cases}
  n\,m \quad
  \textbf{if}\ n \dblcolon \alpha\sto\beta\st m \dblcolon \alpha \\
  \l k \dt m\,\p{\l x \dt n\,\p{\l f \dt k\,\p{x \bsl f}}} \quad 
  \textbf{otherwise}
\end{cases}
\end{aligned}
$

In [6]:
type family f :\: g where
  a :\: (a -> b) = b
  (K c d x) :\: (K d e f) = K c e (x :\: f)

infixr 9 ~\~
class AppL f g where
  (~\~) :: f -> g -> f :\: g
instance AppL a (a -> b) where
  x ~\~ f = f x
instance (d ~ d', AppL in1 in2) => AppL (K c d in1) (K d' e in2) where
  mx ~\~ mf = Tower $ \k -> runTower mx (\x -> runTower mf (\f -> k (x ~\~ f)))

#### Recursive predicate modification

$
\begin{aligned}
m \msl n
&\coloneq
\begin{cases}
  \l x \dt m\,x \land n\,x \quad
  \textbf{if}\ m \dblcolon \alpha\sto t\st n \dblcolon \alpha\sto t \\
  \l k \dt m\,\p{\l p \dt n\,\p{\l q \dt k\,\p{p \msl q}}} \quad 
  \textbf{otherwise}
\end{cases}
\end{aligned}
$

In [7]:
type family f :|: g where
  (a -> T) :|: (a -> T) = a -> T
  (K c d f) :|: (K d e g) = K c e (f :|: g)

infixr 9 ~|~
class AppC f g where
  (~|~) :: f -> g -> f :|: g
instance AppC (a -> T) (a -> T) where
  f ~|~ g = \x -> f x && g x
instance (d ~ d', AppC in1 in2) => AppC (K c d in1) (K d' e in2) where
  mf ~|~ mg = Tower $ \k -> runTower mf (\f -> runTower mg (\g -> k (f ~|~ g)))

To help the type-checker resolve some of the rampant polymorphism, the following
(non-recursive) special cases of $\fsl$ and $\bsl$ are defined

In [8]:
mf </> mx = Tower $ \k -> runTower mf (\f -> runTower mx (\x -> k (f x)))
mf <//> mx = Tower $ \k -> runTower mf (\f -> runTower mx (\x -> k (f </> x)))
mx <\> mf = Tower $ \k -> runTower mx (\x -> runTower mf (\f -> k (f x)))
mx <\\> mf = Tower $ \k -> runTower mx (\x -> runTower mf (\f -> k (x <\> f)))

### Evaluation

$
\begin{aligned}
m^{\low}
&\coloneq
\begin{cases}
  m\,\eta \quad 
  \textbf{if}\ m \dblcolon \mtype{K}\,{\rho}\,\p{\mtype{D}\,{\alpha}}\,{\alpha}\\
  m\,\p{\l n\dt n^\low} \quad
  \textbf{otherwise}
\end{cases}
\end{aligned}
$

In [9]:
class Low r o a where
  lowr :: K r o a -> r
instance Low r (D a) a where
  lowr t = runTower t return
instance (Low r' o' a, o ~ r') => Low r o (K r' o' a) where
  lowr t = runTower t lowr

Note that any given application of $\low$ is equivalent to one of

$
\begin{gathered}
\p{\eta\,\eta}^{\bind}\\
\p{\eta\,\p{\eta\,\eta}^{\bind}}^{\bind}\\
\p{\eta\,\p{\eta\,\p{\eta\,\eta}^{\bind}}^{\bind}}^{\bind}\\
\dots
\end{gathered}
$

### Other convenience functions

$
\begin{aligned}
x^{\lift}
&\coloneq
\p{\eta\,x}^{\bind}
%
&
%
x^{\lift\lift}
&\coloneq
\p{\eta\,\p{\eta\,x}^{\bind}}^{\bind}
%
\\
%
&\ppt=
\l k\dt k\,x
%
&
%
&\ppt=
\l \gamma\dt \gamma\,\p{\l k\dt k\,x}
\end{aligned}
$

In [10]:
u :: a -> K (D r) (D r) a
u x = Tower (return x >>=)

uu :: a -> K (D r') (D r') (K (D r) (D r) a)
uu = u . u

$
\begin{aligned}
m^{\ilow}
&\coloneq
\l k\dt m\,\p{\l n\dt k\,n^{\low}}
\end{aligned}
$

In [11]:
ilowr :: Low r o a => K (D r') o' (K r o a) -> K (D r') o' r
ilowr = (u lowr </>)

$
\begin{aligned}
m^{\reset}
&\coloneq
\p{m^{\low}}^{\bind}
\end{aligned}
$

In [12]:
reset :: (r ~ D b, Low r o a) => K r o a -> K (D r') (D r') b
reset t = Tower (lowr t >>=)